In [84]:
import numpy as np

grav = 9.806
grav = 9.806
Cp   = 1004.
Rgas = 287.04

#-----------------------------------------------------------------------------
# subroutines for grid initialization
#
def zgrid(nz, height = 20000., stag=False): 
    dz = height / float(nz)
    if stag:
        dz = height / float(nz-1)
        return np.arange(nz)*dz
    else:
        return (np.arange(nz)+0.5)*dz

#-----------------------------------------------------------------------------
#
def sounding(gz, dz, theta_sfc = 300., qv_sfc = 14.):
    
    #define parameters
    z_trop     = 12000.
    theta_trop = 343.
    temp_trop  = 213.
    
    # compute theta profile
    theta = np.zeros_like(gz)
    t1    = theta_sfc + (theta_trop - theta_sfc)*((gz/z_trop)**1.25)
    t2    = theta_trop * np.exp(grav * (gz - z_trop) / (Cp * temp_trop))   
    theta = np.where(gz <= z_trop, t1, t2)
    
    # compute rh profile
    rh    = np.where(gz <= z_trop, 1.0 - 0.75*((gz/z_trop)**1.25), 0.25)
    
    # integrate hydrostatic eq
    pi    = np.zeros_like(gz)
    p     = np.zeros_like(gz)
    
    if gz[0] > 1.:
        pi[0] = 1.0 - 0.5*dz[0]*grav/(Cp*theta_sfc)
    
        for k in np.arange(1,gz.shape[0]):
            pi[k] = pi[k-1] - 2.0*dz[k]*grav/(Cp*(theta[k-1]+theta[k]))
    else:
        pi[0] = 1.0
        for k in np.arange(1,gz.shape[0]):
            pi[k] = pi[k-1] - 2.0*dz[k-1]*grav/(Cp*(theta[k-1]+theta[k]))
    
    # compute pressure (Pa) profile
    p = 1.0e5 * pi**(3.5088)
    t = pi*theta - 273.16
    
    # compute qv profile
    qvs = (380./p) * np.exp(17.27*((pi*theta - 273.16)/(pi*theta-36.)))
    qv  = qvs*rh
    qv  = np.where( qv > qv_sfc/1000., qv_sfc/1000., qv)
    
    # recompute RH so that it reflects the new qv_sfc limit - needed to dewpoint computation.
    rh  = qv/qvs
    
    ess = 6.112 * np.exp(17.67 * t / (t + 243.5))
    val = np.log(rh * ess/6.112)
    td  = 243.5 * val / (17.67 - val)
    
    den = p / (t+273.16 * Rgas)
    
    return pi, theta, qv, p, t, td, den

#-----------------------------------------------------------------------------
#
def shear(gz, type=1, shear=12.5, depth=2500.):

    if type == 0.0:
        return np.zeros_like(gz), np.zeros_like(gz)
    
    if type == 1:  # 1D WK squall line or supercell shear...
        
        scale = gz / depth
        
        u = np.where(scale <= 1.0, shear*scale, shear)
        
        return u, np.zeros_like(gz)

#-----------------------------------------------------------------------------
#
def write_2_file(p0, th0, q0, z, theta, qv, u, v, filename='wk.sounding'):
    
    print(filename)
    
    with open(filename, 'w') as f:
        f.write("%12.4f %12.4f  %12.4f \n" % (p0, th0, q0))
        for k in np.arange(z.shape[0]):
            f.write("%12.4f  %12.4f  %12.4f  %12.4f  %12.4f\n" % (z[k], theta[k], 1000.*qv[k], u[k], v[k]))
        
    f.close()


In [85]:
# Create grid
gze = zgrid(201, height = 20000., stag=True)
gzc = zgrid(200, height = 20000.)
dz  = gze[1:] - gze[:-1]

# create soundings

shear0 = 12.5

u, v = shear(gz, shear = shear0, type=1)

qv0 = [11.0,12.0,13.0,14.0,15.0,16.0]

for q in qv0:
    
    pi, theta, qv, p, t, td, den = sounding(gze, dz, theta_sfc=300.5, qv_sfc=q)
    
    write_2_file(1000., 300.5, q, gze, theta, qv, u, v, filename = "input_sounding_qv%2.2i_us%4.1f" % (q, shear0))


input_sounding_qv11_us12.5
input_sounding_qv12_us12.5
input_sounding_qv13_us12.5
input_sounding_qv14_us12.5
input_sounding_qv15_us12.5
input_sounding_qv16_us12.5
